# Правила видимости переменных

In [1]:
def f1(a):
    print(a)
    print(b)

In [2]:
f1(2)

2


NameError: name 'b' is not defined

In [3]:
b = 3

In [4]:
f1(2)

2
3


In [7]:
b1 = 3
def f2(a1):
    print(a1)
    print(b1)
    b1 = 9

In [8]:
f2(2)

2


UnboundLocalError: local variable 'b1' referenced before assignment

In [10]:
def f3(a1):
    global b1
    print(a1)
    print(b1)
    b1 = 9

f3(2)

2
3


In [11]:
b1

9

In [12]:
f3(1)

1
9


In [13]:
from dis import dis

In [14]:
dis(f1)

  2           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  3           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE


In [15]:
dis(f2)

  3           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a1)
              4 CALL_FUNCTION            1
              6 POP_TOP

  4           8 LOAD_GLOBAL              0 (print)
             10 LOAD_FAST                1 (b1)
             12 CALL_FUNCTION            1
             14 POP_TOP

  5          16 LOAD_CONST               1 (9)
             18 STORE_FAST               1 (b1)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


In [16]:
dis(f3)

  3           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a1)
              4 CALL_FUNCTION            1
              6 POP_TOP

  4           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b1)
             12 CALL_FUNCTION            1
             14 POP_TOP

  5          16 LOAD_CONST               1 (9)
             18 STORE_GLOBAL             1 (b1)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


# Замыкания

### Реализация на классах

In [2]:
class Averager():

    def __init__(self):
        self.series = []

    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)

In [3]:
avg = Averager()
avg(10)

10.0

In [4]:
avg(11)

10.5

In [5]:
avg(12)

11.0

### С импользованием фун-ии высокого порядка

In [6]:
def make_averager():
    series = []

    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)

    return averager

In [13]:
avg = make_averager()
avg(10)

10.0

In [14]:
avg(11)

10.5

In [15]:
avg(12)

11.0

In [16]:
avg.__code__.co_varnames

('new_value', 'total')

###

In [17]:
avg.__code__.co_freevars

('series',)

In [22]:
avg.__closure__

(<cell at 0x7fc295cffd30: list object at 0x7fc273652e40>,)

In [23]:
avg.__closure__[0].cell_contents

[10, 11, 12]

## Объявление nonlocal
Пример выше, некорректен, тк все значения хранятся во временнном ряде, и сумма вычисляется при каждом вызове "average". Лучше и правильнее, было бы хранить предидущую сумму и колличество элементов, тогда, когда знаю эти два числа, можно вычислить новое среднее

In [25]:
def make_averager():
    count = 0
    total = 0

    def averger(new_value):
        count += 1
        total += new_value
        return total / count

    return averger

In [26]:
avg = make_averager()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

Ранне с областью видимости все  было в порядке, тк мы не присваивали никакие значения, мы лишь вызывали 'series.append' и передавали ее функциям sum и len. То есть воспользовались тем что список - изменяемый тип.

Однако переменные  неизменяемых типов - числа, строки, кортежи и т.д. - разрешается только читать, но не изменять. При попытке это сделать, неявно создается локальная переменная. Она уже не является свободной и потому не запоминается в замыкании

### Вычисление накопительного среднего без хранения всей истории (исправленный вариант)

In [27]:
def make_averager():
    count = 0
    total = 0

    def averger(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count

    return averger

In [28]:
avg = make_averager()
avg(10)

10.0

In [29]:
avg(11)

10.5

In [30]:
avg(12)

11.0